In [1]:
import pygame
import pygame.freetype
import sys

from pygame.locals import *

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
pygame.init()
pygame.font.init()
font = pygame.font.SysFont('Comic Sans MS', 12)
pygame.display.set_caption("Observable Task")

In [3]:
WINDOW_SIZE = (1500,300)
screen = pygame.display.set_mode(WINDOW_SIZE,0,32)
display = pygame.Surface((750,140)) 
moving_right = False
moving_left = False
not_done = True
vertical_momentum = 0
air_timer = 0
FPS = 144
player_movement = [0,0]
clock = pygame.time.Clock()
position = 0

In [4]:
game_map = [['1','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','1'],
            ['1','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','1'],
            ['1','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','1'],
            ['1','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','1'],
            ['1','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','1'],
            ['1','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','1'],
            ['1','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','0','1'],
            ['1','0','0','0','1','0','0','0','1','0','0','0','1','0','0','0','1','0','0','0','1','0','0','0','1','0','0','0','1','0','0','0','1','0','0','0','1','0','0','0','1'],
            ['1','1','1','1','1','1','1','1','1','1','1','1','1','1','1','1','1','1','1','1','1','1','1','1','1','1','1','1','1','1','1','1','1','1','1','1','1','1','1','1','1','1','1','1','1','1','1']]

In [5]:
positions = [(30, 115), (80, 115)]

In [6]:
dirt_img = pygame.image.load('images/dirt.png')
player_img = pygame.image.load('images/player.png').convert()
player_img.set_colorkey((255,255,255))

In [7]:
player_rect = pygame.Rect(positions[position][0],positions[position][0],5,13)

In [8]:
def collision_test(rect,tiles):
    hit_list = []
    for tile in tiles:
        if rect.colliderect(tile):
            hit_list.append(tile)
    return hit_list

In [9]:
def move(rect,movement,tiles):
    collision_types = {'top':False,'bottom':False,'right':False,'left':False}
    rect.x += movement[0]
    hit_list = collision_test(rect,tiles)
    for tile in hit_list:
        if movement[0] > 0:
            rect.right = tile.left
            collision_types['right'] = True
        elif movement[0] < 0:
            rect.left = tile.right
            collision_types['left'] = True
    rect.y += movement[1]
    hit_list = collision_test(rect,tiles)
    for tile in hit_list:
        if movement[1] > 0:
            rect.bottom = tile.top
            collision_types['bottom'] = True
        elif movement[1] < 0:
            rect.top = tile.bottom
            collision_types['top'] = True
    return rect, collision_types

In [10]:
def create_map():
    tile_rects = []
    y = 0
    for layer in game_map:
        x = 0
        for tile in layer:
            if tile == '1':
                display.blit(dirt_img,(x*16,y*16))
            if tile != '0':
                tile_rects.append(pygame.Rect(x*16,y*16,16,16))
            x += 1
        y += 1
    return tile_rects

In [11]:
def movement():
    global player_movement
    global vertical_momentum
    player_movement = [0,0]
    if moving_right == True:
        player_movement[0] += 2
    if moving_left == True:
        player_movement[0] -= 2
    player_movement[1] += vertical_momentum
    vertical_momentum += 0.2
    if vertical_momentum > 3:
        vertical_momentum = 3

In [12]:
def gravity():
    global air_timer
    global vertical_momentum
    if collisions['bottom'] == True:
        air_timer = 0
        vertical_momentum = 0
    else:
        air_timer += 1

In [13]:
def check_keys(event):
    global not_done
    global moving_right
    global moving_left
    global air_timer
    global vertical_momentum
    if event.type == QUIT:
        not_done = False
    if event.type == KEYDOWN:
        if event.key == K_RIGHT:
            moving_right = True
        if event.key == K_LEFT:
            moving_left = True
        if event.key == K_UP:
            if air_timer < 6:
                vertical_momentum = -5
    if event.type == KEYUP:
        if event.key == K_RIGHT:
            moving_right = False
        if event.key == K_LEFT:
            moving_left = False

In [14]:
while not_done:
    display.fill((146,244,255))

    tile_rects = create_map()
    [display.blit(font.render('State {0}'.format(i), False, (0, 0, 0)), (20+i*50,70)) for i in range(10)]
    movement()
    player_rect, collisions = move(player_rect,player_movement,tile_rects)
    position = 0
    player_rect = pygame.Rect(positions[position][0],positions[position][1],5,13)
    gravity()
    display.blit(player_img,(player_rect.x,player_rect.y))

    for event in pygame.event.get():
        check_keys(event)
        
    screen.blit(pygame.transform.scale(display,WINDOW_SIZE),(0,0))
    pygame.display.update()
    pygame.display.flip()
    clock.tick(FPS)
pygame.quit()